# Export FisherCl results in format for reading by CosmicFish plotting library

In [1]:
import crosspower as cp
import FisherCl as fcl # branch quickCl
#import camb

# set plotting to be in the notebook instead of its own qt window
%matplotlib inline

In [2]:
# load saved Fobj
# get lensing and galaxy Fisher matrix

import FisherCl as fcl
import crosspower as cp
import pickle

#filehandler = open('Fisher_4_1Y.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000; nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# this one used fiducial params and values to match CMB-S4 Science book verification table 8.1

#filehandler = open('Fisher_7_6X.obj','r') # AccuracyBoost=3, lmax=2000, nz=3e5; nonlinear=False,
#neutrino_hierarchy = 'normal', usePrimaryCMB = False; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True

#filehandler = open('Fisher_7_6X_n1.obj','r') # AccuracyBoost=3, lmax=2000, nz=3e5; nonlinear=False,
#neutrino_hierarchy = 'normal', usePrimaryCMB = False; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# n1 for noise1; has kk noise but no gg

filehandler = open('Fisher_7_6X_n2.obj','r') # AccuracyBoost=3, lmax=2000, nz=3e5; nonlinear=False,
#neutrino_hierarchy = 'normal', usePrimaryCMB = False; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# n2 for noise2; has kk noise and gg


Fobj = pickle.load(filehandler) 


lmin = Fobj.lmin
lmax = Fobj.lmax
nMaps = Fobj.nBins + 1
nCosParams = Fobj.nParams - Fobj.nBins
ells = Fobj.ells
obsNames = Fobj.obsNames
paramList = Fobj.paramList

print Fobj.obsList
print Fobj.obsNames
print Fobj.paramList


[ 0.  1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 13. 16. 17. 18. 19. 20.
 24. 25. 26. 27. 32. 33. 34. 40. 41. 48.]
['k,k', 'k,g1', 'k,g2', 'k,g3', 'k,g4', 'k,g5', 'k,g6', 'g1,g1', 'g1,g2', 'g1,g3', 'g1,g4', 'g1,g5', 'g1,g6', 'g2,g2', 'g2,g3', 'g2,g4', 'g2,g5', 'g2,g6', 'g3,g3', 'g3,g4', 'g3,g5', 'g3,g6', 'g4,g4', 'g4,g5', 'g4,g6', 'g5,g5', 'g5,g6', 'g6,g6']
['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'w', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6']


In [3]:
# that Fobj has no FijTE, so load that one here:
#Fij_TE = np.loadtxt('Fij_TE2.dat')  #only TE, with noise
Fij_TE = np.loadtxt('Fij_TEk.dat')  #with k  # no fsky applied # uses H0,not thetaMC
print Fij_TE

[[ 5.33380074e+10 -2.47071699e+10  2.08349036e+06 -4.84348021e+17
  -3.57355218e+08 -7.14692899e+08  2.81158706e+08  5.01249395e+08]
 [-2.47071699e+10  2.40617447e+10 -1.25868675e+07  7.11684914e+17
   1.04921856e+09  7.97406908e+07 -4.00674445e+08 -5.65024890e+08]
 [ 2.08349036e+06 -1.25868675e+07  1.15021430e+04 -4.79005022e+14
  -8.40154387e+05  2.18483710e+05  2.69153462e+05  3.32078185e+05]
 [-4.84348021e+17  7.11684914e+17 -4.79005022e+14  2.37991651e+25
   3.70825142e+16 -4.62891359e+15 -1.31935428e+16 -1.75564716e+16]
 [-3.57355218e+08  1.04921856e+09 -8.40154387e+05  3.70825142e+16
   7.61059412e+07 -1.58113362e+07 -2.05418722e+07 -2.63796070e+07]
 [-7.14692899e+08  7.97406908e+07  2.18483710e+05 -4.62891359e+15
  -1.58113362e+07  1.99954722e+07  1.81219208e+06  6.44740233e+04]
 [ 2.81158706e+08 -4.00674445e+08  2.69153462e+05 -1.31935428e+16
  -2.05418722e+07  1.81219208e+06  7.41472154e+06  9.87134531e+06]
 [ 5.01249395e+08 -5.65024890e+08  3.32078185e+05 -1.75564716e+16
  -

In [4]:
#print Fobj.Fij

In [5]:
# convert As to 10^-9 As.  This is necessary for the CosmicFish library to work properly
AsIndex = 3
Fij_kg = Fobj.Fij
Fij_kg[AsIndex,:] *= 1e-9
Fij_kg[:,AsIndex] *= 1e-9
Fij_TE[AsIndex,:] *= 1e-9
Fij_TE[:,AsIndex] *= 1e-9

In [6]:
# convert Fij_kg from cosmomc_theta to H0
# dthetas calculated in 'Rebuild covar with noise'
dthetas = [-0.0248396985185, 0.0104553863834, 2.96219516316e-05, 0.0, 0.0, 0.0, 
           0.00019347, 0.00093481, # mnu,w
           0.0, 0.0, 0.0, 0.0, 0.0, 0.0] # these for bin biases.  confirm they are 0, please.
# The only derivatives in the Jacobian are on the dtheta/dparam row.
mcIndex = 2
jacobian = np.diag(np.ones(Fobj.nParams))
jacobian[mcIndex,:] = dthetas#*100
#jacobian[:,mcIndex] = dthetas#*100

# multiply these all by d{100thetamc}/d{thetamc} = 100
#jacobian[mcIndex,:] *= 100
#jacobian[:,mcIndex] *= 100

print jacobian

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.48396985e-02  1.04553864e-02  2.96219516e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.93470000e-04  9.34810000e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  

In [7]:
# jacobian-it-up
Fij_kg = np.dot(jacobian.T,np.dot(Fij_kg,jacobian))
print Fij_kg
paramList[2] = 'H0'
print paramList

[[ 3.86080568e+09 -2.73110390e+09 -3.39093176e+05 -8.93106444e+07
  -3.19771041e+08  3.06059651e+05  7.13234091e+07  4.96129664e+06
  -4.10262647e+07 -3.80602444e+07 -2.65855334e+07 -1.76367713e+07
  -1.09242047e+07 -2.00538521e+07]
 [-2.73110390e+09  2.07150129e+09  1.15303591e+05  6.65375445e+07
   2.53099244e+08 -2.32091632e+05 -5.32596343e+07 -9.58331691e+06
   3.72161805e+07  3.22149357e+07  2.16795903e+07  1.42577345e+07
   8.75021046e+06  9.79202783e+06]
 [-3.39093176e+05  1.15303591e+05  1.77198815e+02  5.65308107e+03
   1.35822737e+03 -1.63740634e+01 -4.35009215e+03  8.54867305e+03
  -4.76484149e+03 -1.15773490e+02  1.28224743e+03  1.08166589e+03
   8.37201173e+02  4.84007493e+03]
 [-8.93106444e+07  6.65375445e+07  5.65308107e+03  2.18323860e+06
   7.81570012e+06 -7.56487926e+03 -1.73508697e+06 -1.80930366e+05
   1.06071382e+06  9.98331747e+05  7.17572850e+05  4.91472854e+05
   3.14180096e+05  3.44810040e+05]
 [-3.19771041e+08  2.53099244e+08  1.35822737e+03  7.81570012e+06
  

In [8]:
# setup for saving files in directory structure
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [9]:
#dirName = 'Z_test_2n2' # uses TE with noise, no k
dirName = 'Z_test_2n3' # uses TEk with noise
createFolder(dirName)

createFolder(dirName+'/parameters_analysis')
createFolder(dirName+'/raw_results')
createFolder(dirName+'/results')

In [10]:
# get binBs
binBs = Fobj.binBs
print binBs

[1.27067632 1.69710864 2.3610698  2.91412519 3.65051625 5.32683556]


In [11]:
# create combined Fisher matrix
#Fij_kg = Fobj.Fij
Fij_kgTE = Fij_kg.copy()
Fij_kgTE[:nCosParams,:nCosParams] += Fij_TE 


In [12]:
# fill the raw_results directory
"""
os.chdir(dirName+'/raw_results')

np.savetxt('1_'+dirName+'_fisher_matrix_Fijkg_cls.dat',Fij_kg)
np.savetxt('2_'+dirName+'_fisher_matrix_FijTE_cls.dat',Fij_TE)
np.savetxt('3_'+dirName+'_fisher_matrix_FijkgTE_cls.dat',Fij_kgTE)

# add some code to do the .paramnames files too... but not now

os.chdir('../..')
"""

"\nos.chdir(dirName+'/raw_results')\n\nnp.savetxt('1_'+dirName+'_fisher_matrix_Fijkg_cls.dat',Fij_kg)\nnp.savetxt('2_'+dirName+'_fisher_matrix_FijTE_cls.dat',Fij_TE)\nnp.savetxt('3_'+dirName+'_fisher_matrix_FijkgTE_cls.dat',Fij_kgTE)\n\n# add some code to do the .paramnames files too... but not now\n\nos.chdir('../..')\n"

# Try a version where I put increasing priors on mnu

In [17]:
print paramNames

['ombh2', 'omch2', 'H0', 'As', 'ns', 'tau', 'mnu', 'w', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6']


In [20]:
# define some mnu priors
sigmaMnu1 = 0.03
sigmaMnu2 = 0.01
sigmaMnu3 = 0.003

# copy Fisher matrix
Fij_plus1 = Fij_kgTE.copy()
Fij_plus2 = Fij_kgTE.copy()
Fij_plus3 = Fij_kgTE.copy()

# add priors
mnuIndex = 6
Fij_plus1[mnuIndex,mnuIndex] += sigmaMnu1**-2
Fij_plus2[mnuIndex,mnuIndex] += sigmaMnu2**-2
Fij_plus3[mnuIndex,mnuIndex] += sigmaMnu3**-2

In [22]:
print Fij_kgTE[mnuIndex,mnuIndex]
print Fij_plus1[mnuIndex,mnuIndex]
print Fij_plus2[mnuIndex,mnuIndex]
print Fij_plus3[mnuIndex,mnuIndex]

8804691.939374954
8805803.050486065
8814691.939374954
8915803.050486065


In [23]:
# save Fisher matrices
dirName = 'Z_test_3' # priors on mnu
createFolder(dirName)

createFolder(dirName+'/parameters_analysis')
createFolder(dirName+'/raw_results')
createFolder(dirName+'/results')

os.chdir(dirName+'/raw_results')

np.savetxt('1_'+dirName+'_fisher_matrix_FijkgTE_cls.dat',Fij_kgTE)
np.savetxt('2_'+dirName+'_fisher_matrix_Fij_plus1_cls.dat',Fij_plus1)
np.savetxt('3_'+dirName+'_fisher_matrix_Fij_plus2_cls.dat',Fij_plus2)
np.savetxt('4_'+dirName+'_fisher_matrix_Fij_plus3_cls.dat',Fij_plus3)

# add some code to do the .paramnames files too... but not now

os.chdir('../..')